In [1]:
# Bottom left of interesting area: x,y = (480, 2345)
# Top right of interesting area: x,y = (3341, 327)

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.lines as lines
import matplotlib.patches as patches
import numpy as np

In [2]:
def add_to_plot(fig, ax, img, x, y, section):
    color_str = section[1]
    section_str = section[0]
    
    # Rename on-wedge section
    if section_str == "On wedge":
        section_str = "Wedge"
    
    # Convert data in millimeters to pixels
    x = list(map(lambda x: x*xp/xmm + bl[0], x))
    y = list(map(lambda x: bl[1] - x*yp/ymm, y))

    # Plot data points directly as circles
    for i in range(0,len(x)):
        ax.add_artist(patches.Circle((x[i],y[i]), radius = 10, color = color_str, linewidth = 3))

    # Get the line of best fit
    coeffs = np.polyfit(x, y, 1)
    lin = lambda x: x*coeffs[0] + coeffs[1]

    # Get the two points we'll use to plot the line of best fit

    # Initial guess of the points
    p1 = (480, lin(480))
    p2 = (3341, lin(3341))

    # Modify the points so the line is contained within the figure
    if p1[1] > bl[1]:
        a = np.array([[-1*coeffs[0],1],[0,1]])
        b = np.array([coeffs[1],bl[1]])
        s = np.linalg.solve(a,b)
        p1 = (s[0], s[1])

    if p1[1] < tr[1]:
        a = np.array([[-1*coeffs[0],1],[0,1]])
        b = np.array([coeffs[1],tr[1]])
        s = np.linalg.solve(a,b)
        p1 = (s[0], s[1])

    if p2[1] > bl[1]:
        a = np.array([[-1*coeffs[0],1],[0,1]])
        b = np.array([coeffs[1],bl[1]])
        s = np.linalg.solve(a,b)
        p2 = (s[0], s[1])

    if p2[1] < tr[1]:
        a = np.array([[-1*coeffs[0],1],[0,1]])
        b = np.array([coeffs[1],tr[1]])
        s = np.linalg.solve(a,b)
        p2 = (s[0], s[1])

    # Convert our two points to the format expected
    x = [p1[0], p2[0]]
    y = [p1[1], p2[1]]

    # Plot the line of best fit
    ax.add_artist(lines.Line2D(x, y, linewidth=3, color=color_str, label=section_str))
    return fig, ax, img, p1, p2

In [3]:
def get_seconds_line_numbers(trial, seconds):
    # Open file
    file = open("./trial " + str(trial) + "/raw_data",'r')
    lines = file.readlines()
    
    # Initialize output array, last value is the last line of the file
    arr = [-1,-1,-1,-1,-1,-1,len(lines)-1]
    
    for i in range(0,len(lines)):
        if lines[i].strip() == "10 seconds":
            arr[0] = i;
        elif lines[i].strip() == "11 seconds":
            arr[1] = i;
        elif lines[i].strip() == "12 seconds":
            arr[2] = i;
        elif lines[i].strip() == "13 seconds":
            arr[3] = i;
        elif lines[i].strip() == "14 seconds":
            arr[4] = i;
        elif lines[i].strip() == "15 seconds":
            arr[5] = i;

    file.close()
    
    return arr

def get_data(trial, seconds, field):
    # Output arrays
    x = []
    y = []
    
    # Open file
    file = open("./trial " + str(trial) + "/raw_data",'r')
    lines = file.readlines()
    
    # Get line numbers of each section
    sections = get_seconds_line_numbers(trial, seconds)

    section_line = 0;
    # Read each line in the file until we get to what we're interested in
    for i in range(sections[int(seconds)-10],sections[int(seconds)-10+1]):
        if lines[i].strip() == field:
            section_line = i
            
    # Extract data
    for i in range(section_line+1,len(lines)):
        if lines[i].strip() == "":
            break;
        arr = lines[i].strip().split(" ")
        x.append(float(arr[0]))
        y.append(float(arr[1]))
        
    file.close()

    return x,y

In [ ]:
# Corners of the plot area in pixels
bl = (480, 2345)
tr = (3341, 327)

# Size of the plot area in millimeters (axes units)
xmm = 264.4788
ymm = 185.7814

# Size of the plot area in pixels
xp = np.abs(bl[0]-tr[0])
yp = np.abs(bl[1]-tr[1])

for trial in range(2,11):
    for seconds in range(10,16):
        # Make figure which we will modify
        data_file_str = "./trial " + str(trial) + "/raw_data"
        original_img_str = "./trial " + str(trial) + "/fsss/original/t" + str(trial) + "_" + str("{0:02}".format(seconds)) + ".png"
        fig, ax = plt.subplots(figsize=(20,16), frameon=False, dpi = 300)
        img = mpimg.imread(original_img_str)
        
        # Save slopes of lines of best fit
        left_points = []
        right_points = []

        for section in [("Harmonic edge","darkolivegreen"), ("Soliton edge", "navy"), ("On wedge", "crimson")]:
            x,y = get_data(trial, seconds, section[0])
            fig, ax, img, p1, p2 = add_to_plot(fig, ax, img, x, y, section)
            left_points.append(p1)
            right_points.append(p2)

        # Get angles of the lines of best fit
        # Make slope relative to image
        angles = []
        for i in range(0,len(left_points)):
            angle = np.rad2deg(np.arctan2(left_points[i][1]-right_points[i][1],left_points[i][0]-right_points[i][0]))
            angles.append(180-angle)

        # Save info to a file
        angles_str = "./trial " + str(trial) + "/angles/" + "{0:02}".format(seconds) + ".txt"
        file = open(angles_str, 'w')
        content = []
        content.append("Angles in degrees\n")
        content.append("{0: <31s}: {1}\n".format("Harmonic Edge", str(angles[0])))
        content.append("{0: <31s}: {1}\n".format("Soliton Edge", str(angles[1])))
        content.append("{0: <31s}: {1}\n".format("Wedge edge", str(angles[2])))
        content.append("{0: <31s}: {1}\n".format("|Harmonic edge - Soliton edge|", str(abs(angles[0] - angles[1]))))
        content.append("{0: <31s}: {1}\n".format("|Harmonic edge - Wedge|", str(abs(angles[0] - angles[2]))))
        content.append("{0: <31s}: {1}\n".format("|Soliton edge - Wedge|", str(abs(angles[1] - angles[2]))))
        file.writelines(content)
        file.close()

        # Legend data
        ax.legend(fontsize='xx-large')
        
        plt.axis('off')
        plt.imshow(img)

        # Save the image
        save_img_str = "./trial " + str(trial) + "/fsss/overlay/t" + str(trial) + "_" + str("{0:02}".format(seconds)) + ".png"
        plt.savefig(save_img_str, pad_inches = 0, bbox_inches='tight')
        
        plt.close(fig)

In [ ]:
print(trial)
print(seconds)
print(section)